In [ ]:
from scipy import stats 
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

# Epydemix import
import sys 
sys.path.append("../")
from epydemix.epimodel import EpiModel, simulate
from epydemix.plotting import plot_quantiles, plot_selected_quantiles, plot_posterior, plot_posterior_2d
from epydemix.calibration import calibration_top_perc, calibration_abc_smc
from epydemix.utils import compute_simulation_dates

# Calibration

```epydemix``` currently supports three calibration methods: 
- ```calibration_top_perc```: this methods run a number of simulations and retain the top x%. This is the simpliest method, that allows to get results in the minimum amount of time, since the total number of simulations is set by the user
- ```calibration_rejection``` (not yet implemented): this method implement the simple rejection algorithm.
- ```calibration_abc_smc```: this is the most sophisticated method which implements the ABC-SMC algorithm. This method allows to achieve more accurate results, but it is more computationally expensive.

We import epidemiological data on which we want to calibrate the model. In particular we have two dataset: 
- data_I.csv: here the quantity of intereset is the number of individuals infectious at each time step (prevalence)

- data_incidence.csv: here the quantity of intereset is the number of new individuals infectious at each time step (incidence)

We use these two examples to show the flexibility of pur package to different type of data for calibration.

In [ ]:
df_I = pd.read_csv("./data/data_I.csv")
df_incidence = pd.read_csv("./data/data_incidence.csv")

We first define model and priors, this step is common to all calibration methods:

In [ ]:
# create model 
model = EpiModel(compartments=["S", "I", "R"], population_data_path="../epydemix_data/Indonesia/", population_name="Indonesia")
model.add_parameter(parameters_dict={"beta": 0.04, "mu": 0.2})
model.add_transition(source="S", target="I", rate="beta", agent="I")
model.add_transition(source="I", target="R", rate="mu")

# initial conditions
S=model.population.Nk - np.ones(len(model.population.Nk))
I=np.ones(len(model.population.Nk))

# simulation dates and compute contacts 
simulation_dates = compute_simulation_dates(start_date=df_I.date.values[0], end_date=df_I.date.values[-1], steps="daily")

# parameters
parameters = {"S": S, 
              "I": I, 
              "epimodel": model, 
              "simulation_dates": simulation_dates}

# priors
priors = {"beta": stats.uniform(0.015, 0.025), "mu": stats.uniform(0.15, 0.25)}

## Top % Calibration

In [ ]:
def simulate_wrapper(parameters): 
    results = simulate(**parameters)
    results.update({"data": results["I_total"]})
    return results

results = calibration_top_perc(simulate_wrapper, 
                               parameters=parameters,
                               priors=priors, 
                               data={"data": df_I["I"].values},
                               top_perc=0.05,
                               Nsim=1000)

In [ ]:
plot_selected_quantiles(results, ylabel="I_total")

In [ ]:
plot_posterior(results, "beta", kind="kde", prior_range=False, xlabel="$\\beta$")
plot_posterior(results, "mu", kind="kde", prior_range=False, xlabel="$\\mu$")

In [ ]:
plot_posterior_2d(results, "beta", "mu", kind="kde", prior_range=False, xlabel="$\\beta$", ylabel="$\\mu$")

### Incidence Example

In [ ]:
def simulate_wrapper(parameters): 
    results = simulate(**parameters)
    results.update({"data": np.concatenate(([0], (np.abs(np.diff(results["S_total"])))))})
    return results

results = calibration_top_perc(simulation_function=simulate_wrapper, 
                               parameters=parameters,
                               priors=priors, 
                               data={"data": df_incidence["incidence"].values},
                               top_perc=0.05,
                               Nsim=1000)

In [ ]:
plot_selected_quantiles(results, ylabel="incidence")

In [ ]:
plot_posterior(results, "beta", kind="kde", prior_range=False, xlabel="$\\beta$")
plot_posterior(results, "mu", kind="kde", prior_range=False, xlabel="$\\mu$")

In [ ]:
plot_posterior_2d(results, "beta", "mu", kind="kde", prior_range=False, xlabel="$\\beta$", ylabel="$\\mu$")

## ABC-SMC

In [ ]:
def simulate_wrapper(parameters): 
    results = simulate(**parameters)
    results.update({"data": np.concatenate(([0], (np.abs(np.diff(results["S_total"])))))})
    return results

results = calibration_abc_smc(simulation_function=simulate_wrapper, 
                              priors=priors, 
                              parameters=parameters, 
                              data={"data": df_incidence["incidence"].values},  
                              population_size=50, 
                              max_nr_populations=4)

In [ ]:
plot_selected_quantiles(results, ylabel="incidence")

In [ ]:
plot_posterior(results, "beta", kind="kde", prior_range=False, xlabel="$\\beta$")
plot_posterior(results, "mu", kind="kde", prior_range=False, xlabel="$\\mu$")

In [ ]:
plot_posterior_2d(results, "beta", "mu", kind="kde", prior_range=False, xlabel="$\\beta$", ylabel="$\\mu$")

# Calibrating your own model

In [ ]:
from models.custom_models import my_stochastic_SIR

parameters = {
    'S': 273523000 - 10,
    'I': 10,
    'R': 0,
    'N': 273523000,
    'simulation_dates': simulation_dates,
    'dt': 1.0}

# priors
priors = {"beta": stats.uniform(0.15, 0.35), "mu": stats.uniform(0.15, 0.15)}

In [ ]:
results = calibration_top_perc(simulation_function=my_stochastic_SIR, 
                               parameters=parameters,
                               priors=priors, 
                               data={"data": df_incidence["incidence"].values},
                               top_perc=0.05,
                               Nsim=10000)

In [ ]:
plot_selected_quantiles(results, ylabel="incidence")

# Running Projections

In [ ]:
#results
# run projections

#iterations = 100

#start_date, end_date = pd.to_datetime(start_date), pd.to_datetime(end_date)
#simulation_dates = pd.date_range(start=start_date, end=end_date, freq="d").tolist()
#model.compute_contact_reductions(population, simulation_dates)
